# Практика по кластеризации

#### Настройка среды

- Создаем изолированное окружение: `python -m venv venv`
- Активируем: (unix) `. venv/bin/activate` или (win) `. venv/Scripts/activate`
- Устанавливаем зависимости: `pip install -r practicum_8/requirements.txt`

In [ ]:
import os
import random
import warnings
from collections import Counter
from functools import lru_cache
from os.path import join as pjoin
from typing import Any, Union

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yaml
from ipywidgets import interact, fixed, IntSlider, FloatSlider
from matplotlib import rcParams
from numpy.typing import NDArray
from sklearn.base import TransformerMixin
from sklearn.cluster import DBSCAN, KMeans, MiniBatchKMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler, StandardScaler

%matplotlib inline
rcParams["font.size"] = 14
rcParams["figure.figsize"] = 9, 8

SEED = 42

random.seed(SEED)
np.random.seed(SEED)

### Используемые данные.
Проточная цитометрия — метод исследования дисперсных сред в режиме поштучного анализа элементов дисперсной фазы по сигналам светорассеяния и флуоресценции. Название метода связано с основным приложением, а именно, с исследованием одиночных биологических клеток в потоке.
<img src="misc/cytometry.png" width="680"/>

In [ ]:
with open("../config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

dfs: list[pd.DataFrame] = []
data_dir = cfg["clustering"]["flow_cytometry"]
for file in os.listdir(data_dir):
    df = pd.read_csv(pjoin(data_dir, file))
    df = df.drop(columns="Time-")
    dfs.append(df)

cols = dfs[0].columns
clust_cols = ["FSC-A-", "SSC-A-"]  # 2 основных канала

dfs[0].describe()

In [ ]:
def scatterplot2d(
    df: pd.DataFrame,
    col1: str = "FSC-A-",
    col2: str = "SSC-A-",
    labels: Union[pd.Series, NDArray[np.int_]] = None,
    dots_size: int = 5,
    palette: str = "coolwarm",
) -> None:
    fig, _ = plt.subplots()
    sns.scatterplot(x=df[col1], y=df[col2], hue=labels, s=dots_size, palette=palette)
    fig.canvas.draw()

In [ ]:
scatterplot2d(dfs[0], cols[0], cols[1])

Немножко биологии:
- 2 кластера внизу (видно же, что два!) -- это дебрис и лимфоциты соответственно
- над ними центральный кластер -- моноциты
- огромный кластер наверху -- базофилы + еще что-то (да какая разница?!)

Попробуем тут и далее пытаться в первую очередь выделять лимфоциты. Именно разнообразные субпопуляции лимфоцитов чаще всего выделяют проточным цитометром, которые, в свою очередь, всесторонне характеризуют иммунную систему человека. 

In [ ]:
# избавимся от части выбросов в самом верху
for idx, df in enumerate(dfs):
    mask = (df["FSC-A-"] > 200000) | (df["SSC-A-"] > 240000)
    dfs[idx] = df.drop(df[mask].index)

scatterplot2d(dfs[0], cols[0], cols[1])

### Основные алгоритмы

In [ ]:
def clust_and_viz(
    df: pd.DataFrame,
    clust_cols: list[str],
    clusterer: Any,
    dots_size: int = 5,
    palette: str = "coolwarm",
):
    clusterer.fit(df[clust_cols])
    labels = clusterer.labels_

    print(f"Число кластеров: {len(set(labels))}")

    scatterplot2d(
        df=df,
        col1=clust_cols[0],
        col2=clust_cols[1],
        labels=labels,
        dots_size=dots_size,
        palette=palette,
    )

    plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

    return labels

In [ ]:
# Нормализуем!
df = dfs[0].copy()
df_scaled = df.copy()

scaler = StandardScaler()
df_scaled[cols] = scaler.fit_transform(df_scaled[cols])

#### KMeans

In [ ]:
base_kmeans = KMeans(n_clusters=4, n_init=10)
base_kmeans_labels = clust_and_viz(
    df=df_scaled, clust_cols=clust_cols, clusterer=base_kmeans
)

In [ ]:
def run_elbow_method(
    df: pd.DataFrame,
    clust_cols: list[str],
    max_k: int = 25,
    n_init: int = 10,
    seed: int = SEED,
    plot_results: bool = True,
) -> int:
    w_k = []
    for k in range(1, max_k + 2):
        kmeans = KMeans(n_clusters=k, n_init=n_init, random_state=seed).fit(
            df[clust_cols]
        )
        w_k.append(np.sqrt(kmeans.inertia_))

    # аналитический способ выбора оптимального количества кластеров
    d_k = []
    for idx in range(1, max_k):
        d_k.append(abs(w_k[idx] - w_k[idx + 1]) / abs(w_k[idx - 1] - w_k[idx]))

    if plot_results:
        fig, axs = plt.subplots(1, 2, figsize=(14, 6))
        axs[0].plot(range(1, max_k + 2), w_k, marker="o")
        axs[0].set_xlabel("Число кластеров")
        axs[0].set_ylabel("W(K)")
        axs[1].plot(range(2, max_k + 1), d_k, marker="o")
        axs[1].set_xlabel("Число кластеров")
        axs[1].set_ylabel("D(K)")

    return np.argmax(d_k) + 2


run_elbow_method(df=df_scaled, clust_cols=clust_cols)

In [ ]:
opt_kmeans = KMeans(n_clusters=24, n_init=25)
opt_kmeans_labels = clust_and_viz(
    df=df_scaled, clust_cols=clust_cols, clusterer=opt_kmeans
)

In [ ]:
# Попробуем другой пик из графика метода локтя
opt_kmeans = KMeans(n_clusters=6, n_init=25)
opt_kmeans_labels = clust_and_viz(
    df=df_scaled, clust_cols=clust_cols, clusterer=opt_kmeans
)

#### DBSCAN

In [ ]:
base_dbscan = DBSCAN(eps=0.5, min_samples=5, metric="euclidean", n_jobs=4)

base_dbscan_labels = clust_and_viz(
    df=df_scaled, clust_cols=clust_cols, clusterer=base_dbscan
)

In [ ]:
def plot_sorted_nn_dists(df: pd.DataFrame, min_pts: int = 4) -> None:
    neighbors_fit = NearestNeighbors(n_neighbors=min_pts).fit(df)
    distances, indices = neighbors_fit.kneighbors(df)
    distances = np.sort(distances, axis=0)
    distances = distances[:, 1]

    plt.figure()
    plt.plot(distances)


plot_sorted_nn_dists(df_scaled[clust_cols], min_pts=4)

In [ ]:
opt_dbscan = DBSCAN(eps=0.04, min_samples=3, metric="euclidean", n_jobs=4)

opt_dbscan_labels = clust_and_viz(
    # добавить palette="bright" для лучшей видимости (но легенда становится дискретной)
    df=df_scaled,
    clust_cols=clust_cols,
    clusterer=opt_dbscan,  # , palette="bright"
)

In [ ]:
print(Counter(opt_dbscan_labels))

In [ ]:
# явно выделим лимфоциты
scatterplot2d(df_scaled, clust_cols[0], clust_cols[1], labels=opt_dbscan_labels == 1)

### Интерактив!

Создадим словарик с рассмотренными методами и ограничениями на их основные параметры

In [ ]:
clustering = {
    "dbscan": {
        "method": DBSCAN,
        "params_range": {
            "eps": [*np.arange(0.1, 0.01, -0.01)],
            "min_samples": [*range(25, 0, -1)],
            "metric": ["euclidean", "manhattan"],
            "n_jobs": [*range(1, 5), -1],
        },
    },
    "kmeans": {
        "method": KMeans,
        "params_range": {
            "n_clusters": [*range(2, 31)],
            "n_init": [*range(3, 26)],
            "random_state": fixed(SEED),
        },
    },
    "mbkmeans": {
        "method": MiniBatchKMeans,
        "params_range": {
            "n_clusters": [*range(2, 31)],
            "batch_size": [*range(100, 1001, 100)],
            "n_init": [*range(3, 26)],
            "random_state": fixed(SEED),
        },
    },
}

Напишем класс, который будет кластеризовать данные и отрисовывать результаты в зависимости от поданных в его метод analysis2d параметров. Кэшируем результаты фит-предикта, чтобы не пересчитывать все заново, если, например, изменим размер точек на графике.

In [ ]:
class InteractiveClusterer:
    def __init__(
        self,
        method: Any,
        params_range: dict[str, Any],
        dfs: list[pd.DataFrame],
        scaler: TransformerMixin = StandardScaler(),
    ) -> None:
        self.method = method
        self.clusterer: Any = None
        self.params_range = params_range
        self.dfs = dfs
        self.curr_df = None

    @lru_cache(maxsize=None)
    def fit_predict(
        self,
        patient: int = 0,
        col1: str = "FSC-A-",
        col2: str = "SSC-A-",
        do_scaling: bool = False,
        **kwargs,
    ) -> NDArray[np.int_]:
        self.clusterer = self.method(**kwargs)
        self.clusterer.fit(self.curr_df)
        return self.clusterer.labels_

    def analysis2d(
        self,
        print_clust_num: bool = False,
        dots_size: int = 5,
        palette: str = "coolwarm",
        patient: int = 0,
        col1: str = "FSC-A-",
        col2: str = "SSC-A-",
        do_scaling: bool = True,
        plot_scaled: bool = True,
        **kwargs,
    ) -> None:
        self.curr_df = self.dfs[patient][[col1, col2]].copy()

        if do_scaling:
            self.curr_df[self.curr_df.columns] = scaler.fit_transform(self.curr_df)

        labels = self.fit_predict(
            patient=patient, col1=col1, col2=col2, do_scaling=do_scaling, **kwargs
        )

        if print_clust_num:
            print("Число кластеров:", len(set(labels)))

        scatterplot2d(
            df=self.curr_df if plot_scaled else self.dfs[patient],
            col1=col1,
            col2=col2,
            labels=labels,
            dots_size=dots_size,
            palette=palette,
        )

Немного магии из коробки, юпитеровский виждет: https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html

In [ ]:
method_name = "dbscan"  # kmeans mbkmeans dbscan

params_range = clustering[method_name]["params_range"]

scaler = StandardScaler()  # MinMaxScaler()
clusterer = InteractiveClusterer(**clustering[method_name], dfs=dfs, scaler=scaler)

interact(
    clusterer.analysis2d,
    print_clust_num=True,
    dots_size=[*range(1, 15)],
    palette=["coolwarm", "bright"],
    patient=[*range(0, 5)],
    col1=cols,
    col2=cols,
    do_scaling=[False, True],
    plot_scaled=[False, True],
    **params_range,
)